# Лабораторная работа №2

_Выполнили: Гуревич Михаил, Трохан Александр и Соловьёв Роман, M33001_

In [60]:
from typing import Literal

import json
from itertools import product
from fractions import Fraction

import numpy as np
from simplex_solver import SimplexSolver, LinearProgram, Constraint

## Реализация алгоритма

Реализуем класс игры с нулевой суммой:

In [61]:
class Game:
    def __init__(self, matrix: np.ndarray):
        self.matrix = matrix.copy()
        self.original_matrix = matrix.copy()

        self.remaining_strategies = [list(range(matrix.shape[0])), list(range(matrix.shape[1]))]

    @staticmethod
    def from_json(path: str):
        with open(path) as f:
            matrix = np.array(json.load(f)["payoff_matrix"])
        return Game(matrix)
    
    def __str__(self):
        return str(self.matrix)

    def _eliminate_dominated_by_player(self, matrix: np.ndarray, player: Literal[0, 1]):
        """Eliminate dominated strategies for a given player."""

        dominated_strategies = []

        for i in range(matrix.shape[player]):
            for j in range(matrix.shape[player]):
                if i != j and (
                    (player == 0 and np.all(matrix[i, :] <= matrix[j, :])) or
                    (player == 1 and np.all(
                        matrix[:, i] >= matrix[:, j]))
                ):
                    dominated_strategies.append(i)
                    break

        for i, strategy in enumerate(dominated_strategies):
            del self.remaining_strategies[player][strategy - i]

        return np.delete(matrix, dominated_strategies, axis=player)

    def _eliminate_dominated(self):
        """Eliminate dominated strategies for both players."""

        while True:
            matrix = self.matrix.copy()
            matrix = self._eliminate_dominated_by_player(matrix, 0)
            matrix = self._eliminate_dominated_by_player(matrix, 1)
            if matrix.shape == self.matrix.shape:
                break
            self.matrix = matrix

    def _get_lower_value(self) -> tuple[float, np.ndarray]:
        """Get the lower value of the game and the corresponding strategy."""

        min_ = np.min(self.matrix, axis=1)
        maxmin = np.max(min_)
        return maxmin, np.nonzero(min_ == maxmin)[0]
    
    def _get_upper_value(self) -> tuple[float, np.ndarray]:
        """Get the upper value of the game and the corresponding strategy."""

        max_ = np.max(self.matrix, axis=0)
        minmax = np.min(max_)
        return minmax, np.nonzero(max_ == minmax)[0]
    
    def _get_saddle_point(self) -> tuple[float, tuple[int, int]]:
        """Get the saddle point of the game and the corresponding strategy."""

        maxmin, maxmin_strategies = self._get_lower_value()
        minmax, minmax_strategies = self._get_upper_value()
        if maxmin == minmax:
            for strategy in product(maxmin_strategies, minmax_strategies):
                if self.matrix[strategy] == maxmin:
                    return maxmin, strategy
                
    def _solve_linear_programs(self):
        """Solve primal and dual linear programs associated to the game."""

        positive_matrix = self.matrix.copy()
        if any(positive_matrix.flatten() <= 0):
            positive_matrix -= np.min(positive_matrix) - 1

        primal_constraints = []
        for row in positive_matrix:
            primal_constraints.append(Constraint(row, "leq", 1))
        primal = LinearProgram([1] * positive_matrix.shape[1], "max", primal_constraints)

        dual_constraints = []
        for column in positive_matrix.T:
            dual_constraints.append(Constraint(column, "geq", 1))
        dual = LinearProgram([1] * positive_matrix.shape[0], "min", dual_constraints)

        primal_solution = SimplexSolver(primal).solve()
        dual_solution = SimplexSolver(dual).solve()
        
        if primal_solution[1] != dual_solution[1]:
            raise ValueError("An error occured: the primal and dual solutions have different values.")
        
        return 1 / primal_solution[1], primal_solution[0], dual_solution[0]
                
    def solve_in_pure_strategies(self):
        """Solve the game in pure strategies."""

        self._eliminate_dominated()
        saddle_point = self._get_saddle_point()
        if saddle_point is not None:
            return saddle_point
        else:
            raise ValueError("The game has no saddle point and cannot be solved in pure strategies.")
        
    def solve_in_mixed_strategies(self) -> tuple[float, tuple[tuple[Fraction], tuple[Fraction]]]:
        """Solve the game in mixed strategies."""

        self._eliminate_dominated()
        value, primal_solution, dual_solution = self._solve_linear_programs()
        
        optimal_strategy_player_1 = tuple(np.multiply(dual_solution, value).tolist())
        optimal_strategy_player_2 = tuple(np.multiply(primal_solution, value).tolist())

        real_value = np.dot(np.dot(optimal_strategy_player_1, self.matrix), optimal_strategy_player_2)

        return real_value, (optimal_strategy_player_1, optimal_strategy_player_2)
    
    def get_pure_strategy(self, player: Literal[0, 1], strategy: int) -> np.ndarray[int, int]:
        """Get the pure strategy of a given player."""

        if player == 0:
            return self.original_matrix[self.remaining_strategies[player][strategy], :]
        else:
            return self.original_matrix[:, self.remaining_strategies[player][strategy]].flatten()
        
    def get_mixed_strategy(self, player: Literal[0, 1], strategy: tuple[Fraction]) -> tuple[Fraction]:
        """Get the mixed strategy of a given player."""

        full_strategy: list[Fraction] = []
        j = 0
        for i in range(self.original_matrix.shape[player]):
            if i in self.remaining_strategies[player]:
                full_strategy.append(strategy[j])
                j += 1
            else:
                full_strategy.append(0)
        
        return tuple(full_strategy)

Используя этот класс, реализуем функцию которая будет выводить решение для игры:

In [62]:
def solve(file: str):
    game = Game.from_json(file)
    print("Solving a zero-sum game:")
    print(game)

    try:
        value, strategies = game.solve_in_pure_strategies()

        print(f"The game has a saddle point: {value}")
        print("Pure strategies:")
        print(f"Player 1: {game.get_pure_strategy(0, strategies[0])} (row {game.remaining_strategies[0][strategies[0]]})")
        print(f"Player 2: {game.get_pure_strategy(1, strategies[1])} (column {game.remaining_strategies[1][strategies[1]]})")
    except ValueError:
        print("The game has no saddle point and cannot be solved in pure strategies")
        value, strategies = game.solve_in_mixed_strategies()

        print(f"The game has a value: {value}")
        print("Mixed strategies:")
        print("Player 1:", *game.get_mixed_strategy(0, strategies[0]), sep=", ")
        print("Player 2:", *game.get_mixed_strategy(1, strategies[1]), sep=", ")

## Тестирование алгоритма

С помощью реализованного алгоритма найдём решение для игры со следующей платёжной матрицей:
$$A = \begin{pmatrix}
    3 & 9 & 2 & 1 \\
    7 & 8 & 5 & 6 \\
    4 & 7 & 3 & 5 \\
    5 & 6 & 1 & 7
\end{pmatrix}$$

In [63]:
solve("test1.json")

Solving a zero-sum game:
[[3 9 2 1]
 [7 8 5 6]
 [4 7 3 5]
 [5 6 1 7]]
The game has a saddle point: 5
Pure strategies:
Player 1: [7 8 5 6] (row 1)
Player 2: [2 5 3 1] (column 2)


Теперь проверим игру, для которой не существует решения в чистых стратегиях:
$$A = \begin{pmatrix}
    3 & 4 & 6 & 8 \\
    9 & 10 & 4 & 2 \\
    7 & 7 & 5 & 4
\end{pmatrix}$$

In [64]:
solve("test2.json")

Solving a zero-sum game:
[[ 3  4  6  8]
 [ 9 10  4  2]
 [ 7  7  5  4]]
The game has no saddle point and cannot be solved in pure strategies
The game has a value: 27/5
Mixed strategies:
Player 1:, 2/5, 0, 3/5
Player 2:, 1/5, 0, 4/5, 0


И наконец рассмотрим классическую игру в Камень-Ножницы-Бумага. Её матрица имееет вид:
$$A = \begin{pmatrix}
    0 & 1 & -1 \\
    -1 & 0 & 1 \\
    1 & -1 & 0
\end{pmatrix}$$
Если попытаться воспользоваться алгоритмом без преобразования матрицы, то симлпекс-метод не сойдётся и выдаст ошибку с сообщением о том, что решения нет. Поэтому перед тем, как решать игру в смешанных стратегиях, нужно преобразовать матрицу так, чтобы в ней были только положительные элементы. Тогда отнимем от каждого элемента матрицы минимальный отрицательный элемент - 1. Это преобразование уже реализованно в нашем классе `Game`.

In [65]:
solve("test3.json")

Solving a zero-sum game:
[[ 0  1 -1]
 [-1  0  1]
 [ 1 -1  0]]
The game has no saddle point and cannot be solved in pure strategies
The game has a value: 0
Mixed strategies:
Player 1:, 1/3, 1/3, 1/3
Player 2:, 1/3, 1/3, 1/3
